<a href="https://colab.research.google.com/github/427paul/ai_agent/blob/main/ai_agent_08_Reflection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U "langchain==0.3.*" "langchain-core==0.3.*" "langchain-community==0.3.*" "langgraph==0.3.*" "langchain-huggingface" "huggingface_hub" "sentence-transformers" wikipedia -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflict

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/LangGraph/'

# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

# Reflection

In [ ]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 3-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatOpenAI(model="gpt-4o") # type: ignore
generate = prompt | llm

essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

# Start reflecting on the essay
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

# import asyncio
# from typing import Annotated, List, Sequence
# from langgraph.graph import END, StateGraph, START
# from langgraph.graph.message import add_messages
# from langgraph.checkpoint.memory import MemorySaver
# from typing_extensions import TypedDict


# class State(TypedDict):
#     messages: Annotated[list, add_messages]


# async def generation_node(state: State) -> State:
#     return {"messages": [await generate.ainvoke(state["messages"])]} # type: ignore


# async def reflection_node(state: State) -> State:
#     # Other messages we need to adjust
#     cls_map = {
#         "ai": HumanMessage,
#         "human": AIMessage
#     }
#     # First message is the original user request. We hold it the same for all nodes
#     translated = [state["messages"][0]] + [
#         cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
#     ]
#     res = await reflect.ainvoke(translated)
#     # We treat the output of this as human feedback for the generator
#     return {"messages": [HumanMessage(content=res.content)]}


# builder = StateGraph(State)
# builder.add_node("generate", generation_node)
# builder.add_node("reflect", reflection_node)
# builder.add_edge(START, "generate")


# def should_continue(state: State):
#     if len(state["messages"]) > 6:
#         # End after 3 iterations
#         return END
#     return "reflect"


# builder.add_conditional_edges("generate", should_continue)
# builder.add_edge("reflect", "generate")
# memory = MemorySaver()
# graph = builder.compile(checkpointer=memory)
# # print(graph.get_graph().draw_mermaid())

# config = {"configurable": {"thread_id": "1"}}

# async def main():
#     async for event in graph.astream(
#         {
#             "messages": [
#                 HumanMessage(
#                     content="Generate an essay on the topicality of The Little Prince and its message in modern life"
#                 )
#             ],
#         },
#         config,
#     ):
#         print(event)
#         print("---")

# if __name__ == "__main__":
#     asyncio.run(main())

In [ ]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 3-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="text-generation")
llm = ChatHuggingFace(llm=llm_ep)
generate = prompt | llm

essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

# Start reflecting on the essay
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

# Reflexion

In [ ]:
import datetime
import json

from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from schema import AnswerQuestion, ReviseAnswer, ResponderWithRetries

### Actor ###
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are expert researcher.
            Current time: {time}

            1. {first_instruction}
            2. Reflect and critique your answer. Be severe to maximize improvement.
            3. Recommend search queries to research information and improve your answer.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "\n\n<system>Reflect on the user's original question and the"
            " actions taken thus far. You must respond using the {function_name} function.</reminder>",
        ),
    ]
).partial(
    time=lambda: datetime.datetime.now().isoformat(),
)

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")


validator = PydanticToolsParser(tools=[AnswerQuestion])

initial_answer_chain = actor_prompt_template.partial(
    first_instruction="Provide a detailed ~250 word answer.",
    function_name=AnswerQuestion.__name__,
) | llm.bind_tools(tools=[AnswerQuestion])

first_responder = ResponderWithRetries(
    runnable=initial_answer_chain, validator=validator
)

example_question = "Why is reflection useful in AI?"
initial = first_responder.respond(
    {"messages": [HumanMessage(content=example_question)]}
)
# print(initial["messages"])

"""
{
    "content": "",
    "additional_kwargs": {
        "tool_calls": [
            {
                "id": "call_gRrSeDhRFnkHqhrngCheIT4A",
                "function": {
                    "arguments": {
                        "answer": "Reflection in AI is a critical component for advancing the capabilities and reliability of artificial intelligence systems. One of the primary uses of reflection is enhancing learning and adaptation. When an AI system can reflect on its actions and decisions, it can assess which strategies were successful and which were not, allowing it to adjust its behavior in future tasks effectively, thereby improving its performance over time without explicit external intervention. This could be particularly beneficial in dynamic environments where conditions and requirements change frequently.\n\nAdditionally, reflection allows AI systems to provide explanations for their decisions. This transparency is crucial for building trust with users and for accountability, especially in critical sectors like healthcare or autonomous vehicles, where understanding the rationale behind decisions can prevent errors and improve safety. Through reflection, AI systems can accumulate experiential knowledge, akin to the human learning process, which supports more nuanced and contextually relevant decision-making.\n\nIn terms of development and debugging, reflection is critical for developers as it provides insights into the AI's internal processes, allowing for more targeted and efficient iterations. This capacity for self-assessment can help highlight undesirable biases or inefficiencies, ultimately leading to more robust and unbiased AI systems. Thus, reflection is not just a tool for learning but also a mechanism for ensuring ethical standards and improving system reliability.",
                        "search_queries": [
                            "importance of reflection in AI",
                            "AI systems self-assessment benefits",
                            "AI transparency and accountability through reflection"
                        ],
                        "reflection": {
                            "missing": "Examples of specific AI applications utilizing reflection effectively could reinforce the points made about its importance. Additionally, more technical details on how reflection is implemented in AI models or systems would make the answer more comprehensive.",
                            "superfluous": "The mention of dynamic environments without detailing specific AI use-cases where this matters might be a bit too vague. The reference to human-like learning needs clarification, as it might be too broad of a comparison without substantiation."
                        }
                    },
                    "name": "AnswerQuestion"
                },
                "type": "function"
            }
        ],
        "refusal": null
    },
    "response_metadata": {
        "token_usage": {
            "completion_tokens": 382,
            "prompt_tokens": 231,
            "total_tokens": 613,
            "completion_tokens_details": {
                "accepted_prediction_tokens": 0,
                "audio_tokens": 0,
                "reasoning_tokens": 0,
                "rejected_prediction_tokens": 0
            },
            "prompt_tokens_details": {
                "audio_tokens": 0,
                "cached_tokens": 0
            }
        },
        "model_name": "gpt-4o-2024-08-06",
        "system_fingerprint": "fp_f7a584cf1f",
        "id": "chatcmpl-BNv8p62WPZVm9hTKiO6tX228fBR6m",
        "finish_reason": "tool_calls",
        "logprobs": null
    },
    "id": "run-af355e96-3f65-4b75-b367-f81fe0acee59-0",
    "tool_calls": [
        {
            "name": "AnswerQuestion",
            "args": {
                "answer": "Reflection in AI is a critical component for advancing the capabilities and reliability of artificial intelligence systems. One of the primary uses of reflection is enhancing learning and adaptation. When an AI system can reflect on its actions and decisions, it can assess which strategies were successful and which were not, allowing it to adjust its behavior in future tasks effectively, thereby improving its performance over time without explicit external intervention. This could be particularly beneficial in dynamic environments where conditions and requirements change frequently.\n\nAdditionally, reflection allows AI systems to provide explanations for their decisions. This transparency is crucial for building trust with users and for accountability, especially in critical sectors like healthcare or autonomous vehicles, where understanding the rationale behind decisions can prevent errors and improve safety. Through reflection, AI systems can accumulate experiential knowledge, akin to the human learning process, which supports more nuanced and contextually relevant decision-making.\n\nIn terms of development and debugging, reflection is critical for developers as it provides insights into the AI's internal processes, allowing for more targeted and efficient iterations. This capacity for self-assessment can help highlight undesirable biases or inefficiencies, ultimately leading to more robust and unbiased AI systems. Thus, reflection is not just a tool for learning but also a mechanism for ensuring ethical standards and improving system reliability.",
                "search_queries": [
                    "importance of reflection in AI",
                    "AI systems self-assessment benefits",
                    "AI transparency and accountability through reflection"
                ],
                "reflection": {
                    "missing": "Examples of specific AI applications utilizing reflection effectively could reinforce the points made about its importance. Additionally, more technical details on how reflection is implemented in AI models or systems would make the answer more comprehensive.",
                    "superfluous": "The mention of dynamic environments without detailing specific AI use-cases where this matters might be a bit too vague. The reference to human-like learning needs clarification, as it might be too broad of a comparison without substantiation."
                }
            },
            "id": "call_gRrSeDhRFnkHqhrngCheIT4A",
            "type": "tool_call"
        }
    ],
    "usage_metadata": {
        "input_tokens": 231,
        "output_tokens": 382,
        "total_tokens": 613,
        "input_token_details": {
            "audio": 0,
            "cache_read": 0
        },
        "output_token_details": {
            "audio": 0,
            "reasoning": 0
        }
    }
}
"""

### Revisor ###
revise_instructions = """Revise your previous answer using the new information.
    - You should use the previous critique to add important information to your answer.
        - You MUST include numerical citations in your revised answer to ensure it can be verified.
        - Add a "References" section to the bottom of your answer (which does not count towards the word limit). In form of:
            - [1] https://example.com
            - [2] https://example.com
    - You should use the previous critique to remove superfluous information from your answer and make SURE it is not more than 250 words.
"""

revision_chain = actor_prompt_template.partial(
    first_instruction=revise_instructions,
    function_name=ReviseAnswer.__name__,
) | llm.bind_tools(tools=[ReviseAnswer])
revision_validator = PydanticToolsParser(tools=[ReviseAnswer])

revisor = ResponderWithRetries(runnable=revision_chain, validator=revision_validator)


from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5)

revised = revisor.respond(
    {
        "messages": [
            HumanMessage(content=example_question),
            initial["messages"], # type: ignore
            ToolMessage(
                tool_call_id=initial["messages"].tool_calls[0]["id"], # type: ignore
                content=json.dumps(
                    tavily_tool.invoke(
                        {
                            "query": initial["messages"].tool_calls[0]["args"][ # type: ignore
                                "search_queries"
                            ][0]
                        }
                    )
                ),
            ),
        ]
    }
)

# print(revised["messages"])
"""
{
    "content": "",
    "additional_kwargs": {
        "tool_calls": [
            {
                "id": "call_83iDW7jX12bjAcSL6Ctjk7hY",
                "function": {
                    "arguments": {
                        "answer": "Reflection in AI is a critical component that allows AI systems to evaluate their actions and outcomes, which is crucial for continuous learning and adaptation. This process involves the system analyzing its past decisions, understanding their results, and refining its decision-making strategies to improve future performance. For example, in autonomous driving, reflection enables AI to adjust its algorithms based on previous driving data to enhance safety and efficiency. Furthermore, reflection enhances transparency in AI systems by making decision-making processes clearer to users, which is essential in areas like healthcare and finance.\n\nSelf-reflection is particularly useful in AI models like Reasoning Language Models (RLMs), where it aids in reducing errors and improving reasoning accuracy by self-assessing and providing feedback within the neural network. This process is akin to human metacognition, helping AI to plan and solve problems more effectively by \"thinking about thinking.\" Reflection also plays a role in creative applications like art and music, where AI evaluates past creations to innovate new ones.\n\nIn summary, reflection enhances AI's capability to learn and adapt, supports decision accuracy, increases transparency, and fosters creativity. As AI continues to evolve, integrating reflection will be key to developing robust, trustworthy, and versatile systems.",
                        "search_queries": [
                            "reflection in AI systems",
                            "AI self-reflection examples",
                            "reflection in neural networks"
                        ],
                        "reflection": {
                            "missing": "The previous answer did not include specific AI models or systems that employ reflection. There was also a lack of detailed explanation on how reflection is practically implemented in these systems, such as in neural networks.",
                            "superfluous": "The previous explanation had overly broad descriptions of AI benefits without a direct focus on reflection, and some points were repetitive regarding the general advantages of AI."
                        },
                        "references": [
                            "https://en.wikipedia.org/wiki/Reflection_(artificial_intelligence)",
                            "https://www.psychologytoday.com/us/blog/connecting-with-coincidence/202408/ai-bots-have-some-degree-of-self-reflection",
                            "https://medium.com/stanford-d-school/reflecting-with-ai-a-tool-to-develop-human-intelligence-88cec86babf"
                        ]
                    },
                    "name": "ReviseAnswer"
                },
                "type": "function"
            }
        ],
        "refusal": null
    },
    "response_metadata": {
        "token_usage": {
            "completion_tokens": 448,
            "prompt_tokens": 1928,
            "total_tokens": 2376,
            "completion_tokens_details": {
                "accepted_prediction_tokens": 0,
                "audio_tokens": 0,
                "reasoning_tokens": 0,
                "rejected_prediction_tokens": 0
            },
            "prompt_tokens_details": {
                "audio_tokens": 0,
                "cached_tokens": 0
            }
        },
        "model_name": "gpt-4o-2024-08-06",
        "system_fingerprint": "fp_f5bdcc3276",
        "id": "chatcmpl-BNviv6wdaExzQRsA0nGUCbqS7TdKo",
        "finish_reason": "tool_calls",
        "logprobs": null
    },
    "id": "run-43a20f23-fbd9-4e40-bda9-5651cd75a57d-0",
    "tool_calls": [
        {
            "name": "ReviseAnswer",
            "args": {
                "answer": "Reflection in AI is a critical component that allows AI systems to evaluate their actions and outcomes, which is crucial for continuous learning and adaptation. This process involves the system analyzing its past decisions, understanding their results, and refining its decision-making strategies to improve future performance. For example, in autonomous driving, reflection enables AI to adjust its algorithms based on previous driving data to enhance safety and efficiency. Furthermore, reflection enhances transparency in AI systems by making decision-making processes clearer to users, which is essential in areas like healthcare and finance.\n\nSelf-reflection is particularly useful in AI models like Reasoning Language Models (RLMs), where it aids in reducing errors and improving reasoning accuracy by self-assessing and providing feedback within the neural network. This process is akin to human metacognition, helping AI to plan and solve problems more effectively by \"thinking about thinking.\" Reflection also plays a role in creative applications like art and music, where AI evaluates past creations to innovate new ones.\n\nIn summary, reflection enhances AI's capability to learn and adapt, supports decision accuracy, increases transparency, and fosters creativity. As AI continues to evolve, integrating reflection will be key to developing robust, trustworthy, and versatile systems.",
                "search_queries": [
                    "reflection in AI systems",
                    "AI self-reflection examples",
                    "reflection in neural networks"
                ],
                "reflection": {
                    "missing": "The previous answer did not include specific AI models or systems that employ reflection. There was also a lack of detailed explanation on how reflection is practically implemented in these systems, such as in neural networks.",
                    "superfluous": "The previous explanation had overly broad descriptions of AI benefits without a direct focus on reflection, and some points were repetitive regarding the general advantages of AI."
                },
                "references": [
                    "https://en.wikipedia.org/wiki/Reflection_(artificial_intelligence)",
                    "https://www.psychologytoday.com/us/blog/connecting-with-coincidence/202408/ai-bots-have-some-degree-of-self-reflection",
                    "https://medium.com/stanford-d-school/reflecting-with-ai-a-tool-to-develop-human-intelligence-88cec86babf"
                ]
            },
            "id": "call_83iDW7jX12bjAcSL6Ctjk7hY",
            "type": "tool_call"
        }
    ],
    "usage_metadata": {
        "input_tokens": 1928,
        "output_tokens": 448,
        "total_tokens": 2376,
        "input_token_details": {
            "audio": 0,
            "cache_read": 0
        },
        "output_token_details": {
            "audio": 0,
            "reasoning": 0
        }
    }
}
"""

from langchain_core.tools import StructuredTool
from langgraph.prebuilt import ToolNode

def run_queries(search_queries: list[str], **kwargs):
    """Run the generated queries."""
    return tavily_tool.batch([{"query": query} for query in search_queries])

tool_node = ToolNode(
    [
        StructuredTool.from_function(run_queries, name=AnswerQuestion.__name__),
        StructuredTool.from_function(run_queries, name=ReviseAnswer.__name__),
    ]
)

from typing import Literal

from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


MAX_ITERATIONS = 5
builder = StateGraph(State)
builder.add_node("draft", first_responder.respond)
builder.add_node("execute_tools", tool_node)
builder.add_node("revise", revisor.respond)
# draft -> execute_tools
builder.add_edge("draft", "execute_tools")
# execute_tools -> revise
builder.add_edge("execute_tools", "revise")

# Define looping logic:
def _get_num_iterations(state: list):
    i = 0
    for m in state[::-1]:
        if m.type not in {"tool", "ai"}:
            break
        i += 1
    return i


def event_loop(state: list):
    # in our case, we'll just stop after N plans
    num_iterations = _get_num_iterations(state["messages"]) # type: ignore
    if num_iterations > MAX_ITERATIONS:
        return END
    return "execute_tools"


# revise -> execute_tools OR end
builder.add_conditional_edges("revise", event_loop, ["execute_tools", END])
builder.add_edge(START, "draft")
graph = builder.compile()

events = graph.stream(
    {"messages": [("user", "How should we handle the climate crisis?")]},
    stream_mode="values",
)
for i, step in enumerate(events):
    print(f"Step {i}")
    step["messages"][-1].pretty_print()

"""
Step 0
================================ Human Message =================================

How should we handle the climate crisis?


Step 1
================================== Ai Message ==================================
Tool Calls:
  AnswerQuestion (call_PAAiWjkpv4HjwdraJr91rsLf)
 Call ID: call_PAAiWjkpv4HjwdraJr91rsLf
  Args:
    answer: Addressing the climate crisis requires a multi-faceted approach involving individual, national, and international efforts. Firstly, a key solution lies in transitioning from fossil fuels to renewable energy sources like solar, wind, and hydroelectric power. This transition not only reduces greenhouse gas emissions but also fosters sustainable economic growth and energy security. Secondly, enhancing energy efficiency across all sectors, from industry to homes, can be a significant way to reduce emissions while reducing energy costs.

Moreover, implementing carbon pricing mechanisms such as carbon taxes or cap-and-trade systems can provide economic incentives to reduce emissions and invest in cleaner technologies. It's also imperative to preserve and restore natural carbon sinks like forests and wetlands, which play a crucial role in absorbing carbon dioxide from the atmosphere.

On a political level, international cooperation is vital. Agreements such as the Paris Agreement aim to bring countries together to set and achieve measurable climate goals. On a community and individual level, education and awareness are crucial. Promoting sustainable practices like recycling, reducing food waste, and supporting businesses with green policies can contribute significantly.

Investing in climate-smart agriculture and resilient infrastructure is critical to adapt to climate impacts already being felt. Research and innovation are key; more investment is needed in developing and deploying new technologies that can mitigate climate impacts efficiently.

Overall, handling the climate crisis is a shared responsibility that requires urgent and sustained action across all levels of society and governance.
    search_queries: ['effective strategies for climate change mitigation', 'role of renewable energy in climate change', 'impact of carbon pricing on emissions reduction']
    reflection: {'missing': 'The answer could be improved by providing more specific examples of successful policies and technologies that have been implemented in various countries. Including statistics or case studies would add credibility and depth to the response. Furthermore, the integration of social justice and how vulnerable communities are factored into climate strategies is not addressed.', 'superfluous': 'The mention of the Paris Agreement and its importance might be somewhat redundant as it is commonly recognized in discussions about international climate policy. The overall approach could be more concise in stating strategies, focusing more on action steps and real-world applications rather than broader concepts.'}


Step 2
================================= Tool Message =================================
Name: AnswerQuestion

[[{"title": "Climate Change Mitigation Strategies - Earth@Home", "url": "https://earthathome.org/de/climate-change-mitigation/mitigation-strategies/", "content": "Energy efficiency and energy conservation measures may be the most important and effective mitigation actions we can take. Energy production and use are by far our biggest contributors to greenhouse gas emissions. According to the US Energy Information Administration (eia.gov),\n\nOur success in mitigating climate change will depend on finding ways to use less energy and different energy. This section explores strategies for using less energy.\n\nBuildings [...] Climate change mitigation strategies vary, and they can involve different levels of effort and scope, from broad-ranging actions taken at the government level to actions by specific industries or companies, and to behavior changes made by individuals. A successful mitigation pathway will involve combinations of these strategies.\n\nOn This Page:\n\nRenewable Energy [...] Some forestry options for mitigating the release of carbon are afforestation, reforestation, reducing deforestation, and planting forests that grow rapidly. A report from The Royal Society of the United Kingdom estimated that the combination of these actions could sequester 3 to 18 billion tons of CO₂ from the atmosphere each year. [The Royal Society, 2018] These estimates take into account land availability, economic and ecological constraints, and maintaining land for food and fiber", "score": 0.79732054}, {"title": "Climate Mitigation + Adaptation Strategies - World Resources Institute", "url": "https://www.wri.org/insights/5-strategies-achieve-climate-mitigation-and-adaptation-simultaneously", "content": "Climate actions have often fallen into one of two strategies: mitigation efforts to lower or remove greenhouse gas emissions from the atmosphere, and adaptation efforts to adjust systems and societies to withstand the impacts of climate change. The separation has led to the misinformed view that addressing climate change means pursuing either mitigation or adaptation. [...] It is not always possible or practical to work solely on actions that are both adaptive and mitigating. Nor are these actions a silver bullet to solving the climate crisis. Simply put, where they make sense, governments and communities should pursue such actions. Below are five solutions that can both curb climate change and help us cope with its impacts at the same time:\n\n1. Protect Coastal Wetlands [...] 3. Decentralize Energy Distribution\n\nClimate variability will negatively impact countries’ electricity transmission and distribution infrastructure. At the same time, development and population growth are increasing energy demand and usage. Centralized energy systems — with large power plants and infrastructure connected over long distances — are more vulnerable to climate change since disruptions at one point in the system can affect the entire network.", "score": 0.7931224}, {"title": "Climate change mitigation: reducing emissions", "url": "https://www.eea.europa.eu/en/topics/in-depth/climate-change-mitigation-reducing-emissions", "content": "EU Member States have put in place 3,000 policies and measures to prevent the worst impacts of climate change. National climate change mitigation strategies, policies and other accompanying measures are also in development. These include targets for greenhouse gas emissions in key sectors of the economy, promoting the use of renewable energy and low carbon fuels, energy efficiency improvements in buildings, and many more. [...] Mitigating climate change means reducing the flow of heat-trapping greenhouse gases into the atmosphere. This involves cutting greenhouse gases from main sources such as power plants, factories, cars, and farms. Forests, oceans, and soil also absorb and store these gases, and are an important part of the solution. Reducing and avoiding our emissions requires us to reshape everything we do — from how we power our economy and grow our food, to how we travel and live, and the products we consume.", "score": 0.77096564}, {"title": "Climate change: Strategies for mitigation and adaptation", "url": "https://www.the-innovation.org/article/doi/10.59717/j.xinn-geo.2023.100015", "content": "Transitioning to renewable energy sources and enhancing carbon sinks are crucial steps in mitigating climate change. Adaptation to climate", "score": 0.7576693}], [{"title": "Renewable Energy Sources and Climate Change Mitigation", "url": "https://hepg.hks.harvard.edu/publications/renewable-energy-sources-and-climate-change-mitigation", "content": "This Special Report on Renewable Energy Sources and Climate Change Mitigation (SRREN) impartially assesses the scientific literature on the potential role of renewable energy in the mitigation of climate change for policymakers, the private sector, academic researchers and civil society. It covers six renewable energy sources – bioenergy, direct solar energy, geothermal energy, hydropower, ocean energy and wind energy – as well as their integration into present and future energy systems. It [...] Climate change is one of the great challenges of the 21st century. Its most severe impacts may still be avoided if efforts are made to transform current energy systems. Renewable energy sources have a large potential to displace emissions of greenhouse gases from the combustion of fossil fuels and thereby to mitigate climate change. If implemented properly, renewable energy sources can contribute to social and economic development, to energy access, to a secure and sustainable energy supply,", "score": 0.8925721}, {"title": "The Role of Renewable Energy - EcoMatcher", "url": "https://www.ecomatcher.com/the-role-of-renewable-energy/", "content": "Renewable energy is a critical solution in combating climate change. By transitioning to a low-carbon energy mix, we can reduce greenhouse gas emissions, improve energy security, and provide millions of people access to clean, reliable, and affordable energy. Governments, businesses, and individuals must work together to make the transition to renewable energy a reality and ensure a sustainable future for all.\n\nTags:\n\nRecent Posts\n\nCategories\n\nTags\n\nAbout EcoMatcher [...] As the world moves towards a low-carbon economy, the need for clean and sustainable energy sources has never been greater. Climate change threatens not just the environment but also everything that depends on it: human and animal health, food and water security, and economic stability, to name a few. To address this global challenge, renewable energy has become a critical component in the fight and is beginning to play a central role in transforming our energy systems. [...] The use of renewable energy can significantly reduce the amount of carbon emissions released into the atmosphere and help combat the negative impacts of climate change. By transitioning to a low-carbon energy mix, we can also reduce the risk of extreme weather events. It’s also crucial because it has the potential to provide clean, reliable, and affordable energy to communities that are currently without access to power. This can drive economic growth, create jobs, and improve the standard of", "score": 0.8740022}, {"title": "Renewable Energy Is Key to Fighting Climate Change - NRDC", "url": "https://www.nrdc.org/bio/noah-long/renewable-energy-key-fighting-climate-change", "content": "Renewable energy is one of the most effective tools we have in the fight against climate change, and there is every reason to believe it will succeed. A recent New York Times column seems to imply that renewable energy investments set back efforts to address climate change—nothing could be further from the truth. What’s more, renewable technologies can increasingly save customers money as they displace emissions from fossil fuels. [...] In the longer term, the U.S. Environmental Protection Agency’s Clean Power Plan to establish the first national limits on carbon pollution from power plants will continue to drive renewable energy growth. Wind and solar energy will play a central role in achieving the emissions cuts required, and carbon policies like the Clean Power Plan will be critical to ensuring that low-carbon resources are prioritized over higher-emitting power plants.\nThe benefits are huge [...] In addition to the climate benefits that they will help deliver, renewables already provide a wide range of market and public health benefits that far outweigh their costs. A recent report from the Department of Energy and Lawrence Berkeley National (LBNL) Laboratory found that renewable portfolio standards—state policies that mandate that a specific amount of the state’s electricity comes from renewables—provide a wide range of economic, health, and climate benefits. The report concluded that", "score": 0.83452857}, {"title": "Renewable Energy | MIT Climate Portal", "url": "https://climate.mit.edu/explainers/renewable-energy", "content": "widely viewed as playing a central role in climate change mitigation and a clean energy transition. [...] together they still only provide 9% of electricity worldwide.1 As societies work to lower their greenhouse gas emissions, renewable energy is expected to play a large role, especially if we switch more heating and transportation to run on electric power and solve the problem of affordable, large-scale energy storage. How much of our energy we ultimately get from renewables will also depend on their ability to compete with other low-carbon technologies, such as nuclear, carbon capture and [...] Main navigation\n\nMain navigation\n\nNew to Climate Change?\n\nRenewable Energy\n\nRenewable energy is energy from sources we cannot run out of. Some types of renewable energy, like wind and solar power, come from sources that are not depleted when used. Others, like biomass, come from sources that can be replenished. Common types of renewable energy are wind, solar, hydropower, biomass and geothermal.", "score": 0.81418616}, {"title": "Renewable energy and its importance for tackling climate change", "url": "https://www.nhm.ac.uk/discover/renewable-energy.html", "content": "Replacing fossil fuel-reliant power stations with renewable energy sources, such as wind and solar, is a vital part of stabilising climate change and achieving net zero carbon emissions.\nProfessor Magda Titirici, Chair in Sustainable Energy Materials at Imperial College London, offers an introduction to renewable energy and the future of clean, green power in the UK.\nWhat is renewable energy? [...] Renewable energy comes from sources that replenish naturally and continually within a human lifetime. Renewable energy is often called sustainable energy.\nMajor sources of renewable energy include solar, wind, hydroelectric, tidal, geothermal and biomass energy, which is derived from burning plant or animal matter and waste.\nSwitching our reliance on fossil fuels to renewable energy sources that produce lower or no greenhouse gas emissions is critically important in tackling the climate crisis. [...] One of the main benefits of most renewable energy sources is that they don't release carbon dioxide or pollute the air when they are used to produce electricity or heat. Greenhouse gases are emitted during the lifetime of some of the technologies - for example, during their manufacture or construction - but overall emissions are significantly lower than for fossil fuels.", "score": 0.8036831}], [{"title": "Systematic review and meta-analysis of ex-post evaluations on the ...", "url": "https://www.nature.com/articles/s41467-024-48512-w", "content": "We find consistent evidence that carbon pricing policies have caused emissions reductions. Statistically significant emissions reductions are found for 17 of the reviewed carbon pricing policies, with immediate and sustained reductions of between –5% to –21% (–4% to –15% when correcting for publication bias). Our heterogeneity analysis suggests that differences in estimates from the studies are driven by the policy design and context in which carbon pricing is implemented, while often discussed [...] Our meta-regression results suggest that the policy effectiveness of carbon pricing policies increases with time. Studies covering longer time periods after the introduction of the carbon price report larger emissions reduction effects compared to assessments for shorter time periods. While this finding should be treated with caution, as most of the primary studies assume constant treatment effects for their estimations, it hints towards increasing emissions reductions in the years following [...] higher carbon prices lead to larger emissions reductions within a carbon pricing scheme as commonly found in available assessments of fuel price elasticities24,57,58. In line with this argument, we find that the relationship between carbon price levels and emissions reductions in our meta-analytic framework is dominated by the across-scheme variation in prices, which accounts for 91% of the variation in our dataset while the variation within schemes only accounts for 9%. The interpretation for", "score": 0.9092728}, {"title": "Carbon Pricing and Its Role in Reducing CO2e Emissions", "url": "https://www.terrascope.com/blog/carbon-pricing-and-its-role-in-reducing-co2e-emissions", "content": "In the battle against climate change, finding effective strategies to reduce greenhouse gas emissions has become paramount. Carbon pricing has emerged as a powerful instrument in driving emission reductions. Carbon pricing aims to put a price on carbon emissions, creating economic incentives for businesses and individuals to reduce their carbon footprint. This article delves into the role of carbon pricing in reducing CO2e emissions, exploring its mechanisms, impact on various sectors, [...] The impact of carbon pricing extends across a range of sectors, each contributing to varying extents of global CO2e emissions. In the power sector, carbon pricing transforms the energy landscape by steering investments towards renewable sources such as solar, wind, and hydropower. Fossil fuel-based power generation becomes costlier, prompting a shift towards cleaner alternatives. The industrial sector has been witnessing a shift towards cleaner production methods, energy efficiency [...] Carbon pricing has a significant impact on businesses, as it aims to internalise the external costs of carbon emissions. Businesses face increased operational expenses, which can lead to higher production costs and reduced profit margins, particularly for carbon-intensive industries. On the other hand, it also acts as a catalyst for sustainable practices, incentivising businesses to embrace greener technologies and processes, thereby diminishing their reliance on fossil fuels and minimising", "score": 0.89030397}, {"title": "The economic effects of carbon pricing - CEPR", "url": "https://cepr.org/voxeu/columns/economic-effects-carbon-pricing", "content": "Carbon pricing policies are critical tools to mitigate the effects of climate change. This column examines the impact of European carbon pricing policies on the economy, contrasting the common carbon market with national carbon taxes. The authors find that both policies have effectively reduced emissions, but price changes in the carbon market have been associated with larger economic costs. They also discuss four factors that can account for the differential effects and the impacts of the [...] Carbon pricing policies are increasingly used as a tool to mitigate climate change. While there is mounting evidence on the effectiveness of such policies for emission reductions (e.g. Martin et al. 2014, Andersson 2019), less is known about their economic effects. In this column, we provide new empirical evidence on the aggregate and regional impacts of carbon pricing, drawing on our recent research on the European experience (Känzig and Konradt 2023). [...] more pronounced in countries that do not recycle tax revenues, displaying a stronger fall in output and an increase in unemployment. However, these differential impacts cannot be uniquely attributed to revenue recycling as non-revenue recycling countries also display a somewhat stronger response to energy prices. Interestingly, the emission responses turn out to be comparable, suggesting that redistributing tax revenues can lower economic costs without compromising emission reductions.", "score": 0.80735475}, {"title": "How can we measure the impact of carbon prices on global warming?", "url": "https://www.brookings.edu/articles/how-can-we-measure-the-impact-of-carbon-prices-on-global-warming/", "content": "A higher carbon price tends to reduce greenhouse gas emissions and atmospheric concentrations of CO2 and eventually curb the increase in global", "score": 0.79099923}, {"title": "What is Carbon Pricing?", "url": "https://carbonpricingdashboard.worldbank.org/what-carbon-pricing", "content": "There is a growing consensus among both governments and businesses on the fundamental role of carbon pricing in the transition to a decarbonized economy. For governments, carbon pricing is one of the instruments of the climate policy package needed to reduce emissions. In most cases, it is also  a source of revenue, which is particularly important in an economic environment of budgetary constraints. Businesses use internal carbon pricing to evaluate the impact of mandatory carbon prices on", "score": 0.7325349}]]


Step 3
================================== Ai Message ==================================
Tool Calls:
  ReviseAnswer (call_2aDzUYDQTZ6vVnMRiOKPaqQp)
 Call ID: call_2aDzUYDQTZ6vVnMRiOKPaqQp
  Args:
    answer: Addressing the climate crisis requires comprehensive efforts across various levels. Transitioning to renewable energy sources such as solar, wind, and hydropower is crucial. These sources can significantly reduce greenhouse gas emissions and offer sustainable alternatives to fossil fuels [1]. Implementing energy efficiency measures and reducing energy consumption across industries, buildings, and transportation are also vital steps [2]. Carbon pricing mechanisms, like carbon taxes and cap-and-trade systems, provide strong economic incentives to reduce emissions, with studies showing emissions reductions ranging between 5% to 21% [3]. Furthermore, protecting and restoring natural carbon sinks, including oceans, forests, and wetlands, is essential for sequestering CO2 [2].

International cooperation remains critical. Agreements like the Paris Agreement set joint targets for emissions reductions and foster global collaboration [4]. On a local level, enhancing climate education and encouraging sustainable practices—such as recycling and reducing food waste—can foster significant change. Supporting climate-smart agriculture and investing in resilient infrastructure allow communities to adapt to ongoing climate impacts [2].

Incorporating social equity into climate strategies ensures that vulnerable communities are protected and can share in the benefits of climate actions. Real-world examples and wider public engagement are crucial for transparency and accountability in these initiatives.
    search_queries: ['successful climate change policies by country', 'impact of renewable energy on national emissions', 'carbon pricing effectiveness studies']
    reflection: {'missing': 'Specific examples and more detailed case studies about the effectiveness of suggested measures can strengthen the credibility and provide a clearer picture of viable strategies. Emphasizing the role of social equity and how vulnerable communities are integrated into climate plans can further enhance the answer.', 'superfluous': 'General statements about international cooperation might be reduced or substituted with more specific examples of international partnerships or agreements that have shown measurable success in mitigating climate change impacts.'}
    references: ['https://hepg.hks.harvard.edu/publications/renewable-energy-sources-and-climate-change-mitigation', 'https://earthathome.org/de/climate-change-mitigation/mitigation-strategies/', 'https://www.nature.com/articles/s41467-024-48512-w', 'https://www.eea.europa.eu/en/topics/in-depth/climate-change-mitigation-reducing-emissions']


Step 4
================================= Tool Message =================================
Name: ReviseAnswer

[[{"title": "Climate Policies with Real-World Results", "url": "https://www.worldbank.org/en/news/feature/2023/09/19/climate-policies-with-real-world-results", "content": "Thousands of climate policies have failed to stop climate change, yet some governments have made tangible progress.\nThe World Bank’s Reality Check report highlights 25 successful climate policies from countries as disparate as Egypt, Niger, China, and Peru.\nThe report provides insights on how countries design and implement effective climate policies and on the compromises doing so can require. [...] Yet governments across the globe have made tangible progress and many climate policies have been successfully implemented. A new World Bank report, Reality Check: Lessons from 25 Policies Advancing a Low-Carbon Future showcases examples across sectors and five continents, from countries as disparate as Egypt, Niger, China and Peru. [...] Sri Lanka\nSt. Kitts and Nevis\nSt. Lucia\nSt. Vincent and Grenadines\nSudan\nSuriname\nSwaziland\nSweden\nSwitzerland\nSyria\nعربي\n\n\nTajikistan\nРусский\n\n\nTanzania\nThailand\nภาษาไทย\n\n\nTimor Leste (East Timor)\nTogo\nFrançais\n\n\nTonga\nTrinidad and Tobago\nTunisia\nعربي\nFrançais\n\n\nTurkey\nTurkmenistan\nРусский\n\n\nTuvalu\nUganda\nUkraine\nYкраїнський\n\n\nUnited Arab Emirates\nUnited Kingdom\nUnited States of America\nUruguay\nEspañol\n\n\nUzbekistan\nРусский\n\n\nVanuatu\nVenezuela\nEspañol\n\n\nVietnam\nTiẽng Việt", "score": 0.91760606}, {"title": "How successful are countries with climate change policies?", "url": "https://www.prescouter.com/2024/09/how-successful-are-countries-with-climate-change-policies/", "content": "The analysis shows that Chile, Finland, the Netherlands, Switzerland, Japan, and India had climate policies that provided significant CO2 emission reductions.", "score": 0.8757978}, {"title": "3 Leading Countries in Climate Policy | Earth.Org", "url": "https://earth.org/countries-climate-policy/", "content": "Denmark, Sweden, and Chile are leading countries in climate action. Here's why and what other countries can learn from their climate policy strategies.", "score": 0.71305263}, {"title": "Global Climate Agreements: Successes and Failures", "url": "https://www.cfr.org/backgrounder/paris-global-climate-change-agreements", "content": "However, China and India are now among the world’s top annual emitters, along with the United States. Developed countries have argued that those countries must do more now to address climate change. [...] levels by 2030, and China said it aims to reach peak emissions before 2030. [...] Since 2015, dozens of countries—including the top emitters—have submitted stronger pledges. For example, President Biden announced in 2021 that the United States will aim to cut emissions by 50 to 52 percent compared to 2005 levels by 2030, doubling former President Barack Obama’s commitment. The following year, the U.S. Congress approved legislation that could get the country close to reaching that goal. Meanwhile, the EU pledged to reduce emissions by at least 55 percent compared to 1990", "score": 0.61168414}, {"title": "Top 10 countries leading the world's decarbonization - ClimateTrade", "url": "https://climatetrade.com/top-10-countries-leading-the-worlds-decarbonization/", "content": "3. Denmark\n\nDenmark is working to cut GHG emissions by 70% from 1990 levels by 2030, and to source at least half of its energy needs from renewables by 2030. Traditionally based on oil, natural gas and coal, Denmark’s energy mix has now diversified to include more biofuels, wind and solar generation.\n\n4. Switzerland [...] It is no surprise that Sweden tops the index: the country is aiming to cut greenhouse gas emissions by 59% by 2030 compared with 2005, and to have a net-zero carbon economy by 2045. It was also the first country to introduce carbon pricing and has the highest carbon tax in the world at €122 per ton in 2023.\n\nToday, the bulk of Sweden’s energy comes from nuclear, hydro and biofuels, with an increasing portion of wind and solar.\n\n2. Norway [...] The UK plans to cut greenhouse gas emissions by at least 68% by 2030, from 1990 levels, and to achieve net zero emissions by 2050. In fact, in 2020, the country’s emissions had already dropped by almost 45% compared to 1990. The UK is making progress by reducing its dependence on coal, oil and natural gas, and increasing the amount of biofuels, renewables and nuclear energy it uses.\n\n8. New Zealand", "score": 0.5792344}], [{"title": "Benefits of Renewable Energy Use | Union of Concerned Scientists", "url": "https://www.ucsusa.org/resources/benefits-renewable-energy-use", "content": "For example, a 2009 UCS analysis found that a 25 percent by 2025 national renewable electricity standard would lower power plant CO2 emissions 277 million metric tons annually by 2025—the equivalent of the annual output from 70 typical (600 MW) new coal plants [4]. [...] In addition, a ground-breaking study by the US Department of Energy's National Renewable Energy Laboratory (NREL) explored the feasibility of generating 80 percent of the country’s electricity from renewable sources by 2050. They found that renewable energy could help reduce the electricity sector’s emissions by approximately 81 percent [5].\n\nSign up for UCS email today\n\nGet the latest updates on our work to make clean energy choices and how you can help right from your inbox. [...] In contrast, most renewable energy sources produce little to no global warming emissions. Even when including “life cycle” emissions of clean energy (ie, the emissions from each stage of a technology’s life—manufacturing, installation, operation, decommissioning), the global warming emissions associated with renewable energy are minimal [3].", "score": 0.8364614}, {"title": "Cost, environmental impact, and resilience of renewable energy ...", "url": "https://link.springer.com/article/10.1007/s10311-022-01532-8", "content": "This section examines the impact of renewable energy sources on the environment. The study demonstrates that using renewable energy in most regions and nations positively affects the environment by directly reducing carbon dioxide emissions. However, renewable energy in some remote regions may have adverse environmental impact. To summarize, renewable energy is environmentally friendly and can contribute to sustainable development in the world.\nEconomic impact of renewable energy [...] 46,405 × 103 tons of carbon dioxide, generating a price of $433,427.6 × 103 based on certified emission reduction return and fuel savings would amount to 19,066 kilotons. Furthermore, according to the analysis of the environmental impact of the use of renewable energy in each country in Table 4, we find that the use of renewable energy in BRICS (Brazil, Russia, India, China and South Africa), Europe, the USA, and Japan has a positive impact on the environment, i.e., a reduction in carbon [...] To achieve net-zero emissions by 2050, renewable energy sources must be established by 2030 at the cost of $4 trillion annually. The initial expense may be prohibitive for many nations, which may require financial and technical assistance to complete the energy transition. However, by achieving net-zero carbon emissions, renewable energy investments will be profitable and could save up to $4.2 trillion annually. Additionally, renewables are less susceptible to market shocks and increase each", "score": 0.7975099}, {"title": "The effect of renewable energy on carbon emissions through ...", "url": "https://pmc.ncbi.nlm.nih.gov/articles/PMC10907782/", "content": "The findings suggest that when FDI acts as the intermediary, renewable energy exhibits a negative indirect impact on CO2 emissions.", "score": 0.77137923}, {"title": "Assessing the environmental impacts of renewable energy sources", "url": "https://www.sciencedirect.com/science/article/pii/S0301479723013130", "content": "This study investigates the impact of renewable and non-renewable energy sources on carbon emissions in the context of China's 14th Five-Year Plan (2021–2025).", "score": 0.7311551}, {"title": "4 Environmental Impacts of Renewable Electricity Generation", "url": "https://nap.nationalacademies.org/read/12987/chapter/6", "content": "regard to GHG emissions. When life cycle emissions are considered, all forms of renewable electricity production are expected to have significantly lower GHG emissions per unit of electricity produced than generation from conventional coal and natural gas plants. With the exception of emissions of NOx and carbonaceous materials from biomass combustion, rates of life cycle emissions of conventional air pollutants from renewable electricity generation are also sharply lower than from coal and [...] the two countries make in advancing use of renewable energy will provide global benefits, in terms of enhanced technological understanding, reduced costs through expanded deployment, and reduced greenhouse gas (GHG) emissions relative to conventional generation from fossil fuels. [...] In this section, we present results from published LCA studies compiled by the National Academies Committee on Electricity from Renewable Resources: Status, Prospects, and Impediments (NAS/NAE/NRC, 2010a). These studies provide high-level comparisons of fossil- and renewable-fueled technologies in\n\nterms of net energy production, emissions of GHGs and conventional air pollutants, water use, and land use.", "score": 0.7043437}], [{"title": "Systematic review and meta-analysis of ex-post evaluations on the ...", "url": "https://www.nature.com/articles/s41467-024-48512-w", "content": "Our meta-regression results suggest that the policy effectiveness of carbon pricing policies increases with time. Studies covering longer time periods after the introduction of the carbon price report larger emissions reduction effects compared to assessments for shorter time periods. While this finding should be treated with caution, as most of the primary studies assume constant treatment effects for their estimations, it hints towards increasing emissions reductions in the years following [...] 38 carbon pricing schemes already implemented by 2015, for 18 of these we could not find a single study on effectiveness, despite the broad set of bibliographic databases searched (see Supplementary Information). There is also little evidence on the effectiveness of carbon pricing relative to the level of the carbon price (carbon price elasticity). We identify only nine price elasticity studies, providing too few effect sizes for meta-analysing these separately. [...] In this first quantitative meta-analysis of carbon pricing evaluations, we find robust evidence that existing carbon pricing schemes that have been evaluated to date are effective in reducing GHG emissions. Our machine-learning enhanced approach to study identification finds more than twice as many ex-post evaluations than existing reviews15,25,26,28, studying the effectiveness of 21 carbon pricing policies. Our meta-analysis finds that at least 17 of these policies have caused significant", "score": 0.8803145}, {"title": "Carbon Pricing Studies - Citizens' Climate Lobby", "url": "https://citizensclimatelobby.org/carbon-pricing-studies/", "content": "Carbon Pricing Studies\n\nCarbon Pricing Studies\n\nStudies show that putting a revenue-neutral price on pollution will be good for people, the economy, and the environment.\n\nThese carbon pricing studies show that an economy-wide carbon price with  money allocated to Americans to spend with no restrictions will have positive impacts on America’s climate, health, and pocketbooks. [...] This 2023 study published in the Journal of the Association of Environmental and Resource Economists found that carbon pricing in California reduced other air pollution and improved public health across the state, including in frontline communities.\n\nLearn More:\n\nA carbon price is the single most powerful tool available to reduce America’s carbon pollution. [...] This 2023 study published in the Journal of Cleaner Production found that “the adoption of carbon pricing policies is associated with an increase in patent applications for climate mitigation technologies.” In other words, carbon pricing spurs innovation.", "score": 0.8443195}, {"title": "How robust is the evidence on carbon pricing? - LSE", "url": "https://www.lse.ac.uk/granthaminstitute/news/how-robust-is-the-evidence-on-carbon-pricing/", "content": "The good news is that there is considerable scope for cross-learning across studies to address these methodological issues. Furthermore, a series of recent quasi-experimental studies provide innovative approaches in measuring carbon pricing effects, such as using prospective policies (rather than actual policies) to study effects and new methods that enable the study of carbon pricing effects even where there is limited variation in the data. [...] Yet, there remain three challenges that limit our understanding of carbon pricing effects. First, a large part of the evidence is based on economic models that predict, rather than evaluate, the effect of carbon pricing policies. Second, of the empirical studies that evaluate carbon pricing, there are large differences in findings, depending on the empirical design and framing of carbon pricing. Lastly, the empirical methods that are frequently used, such as cross-sectional regressions, do not [...] Fortunately, there has been a wave of new empirical research that provides useful evidence. This research utilises innovative quasi-experimental designs to examine the economic and environmental effects of carbon pricing. By exploiting changes in policy over time and within a specific geographical area, these studies can examine whether a causal relationship exists between carbon pricing, and emissions and economic outcomes. According to a review I recently conducted, this evidence suggests", "score": 0.8332299}, {"title": "Carbon pricing works, major meta-study finds - Phys.org", "url": "https://phys.org/news/2024-05-carbon-pricing-major-meta.html", "content": "More information:\n\t\t\t\t\t\t\t\t\t\t\t\tNiklas Döbbeling-Hildebrandt et al, Systematic review and meta-analysis of ex-post evaluations on the effectiveness of carbon pricing, Nature Communications (2024). DOI: 10.1038/s41467-024-48512-w\n\n\nJournal information:\nNature Communications\n\nJournal information:\nNature Communications\n\nProvided by\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tMercator Research Institute on Global Commons and Climate Change (MCC)\n\nExplore further\n\nFacebook\n\nTwitter\n\nEmail\n\nFeedback to editors [...] Between 5 and 21% emission reductions: this is the empirically measured effect of carbon pricing systems in their first few years of operation. A research team now identifies these findings for 17 real-world climate policies around the globe, condensing the state of knowledge more comprehensively than ever. The team uses artificial intelligence to collate existing surveys, making them comparable using a novel calculation concept. [...] To date, the empirical data shows, among other things, that the introduction of carbon pricing in some Chinese provinces has had an above-average effect on the emissions balance. In general, the effect tends to be particularly increased by an offensive policy design (\"announcement effect\") and a favorable environment (low CO2 avoidance costs).", "score": 0.7933146}, {"title": "The argument for a carbon price - Our World in Data", "url": "https://ourworldindata.org/carbon-price", "content": "But it is also the case that carbon pricing is not very popular among voters. Study after study finds that a key reason for this is that many do not believe that a price on carbon is actually effective in reducing emissions.17 While it is the effectiveness of carbon prices that makes them popular among experts, it is the perceived ineffectiveness that makes them unpopular among non-experts.", "score": 0.7781275}]]


Step 5
================================== Ai Message ==================================
Tool Calls:
  ReviseAnswer (call_yx0KGQOzJW1EP2JPijcdGkVq)
 Call ID: call_yx0KGQOzJW1EP2JPijcdGkVq
  Args:
    answer: Addressing the climate crisis involves multi-level strategies. Transitioning to renewable energy like solar, wind, and hydropower is essential, as these sources significantly reduce greenhouse gas emissions and provide sustainable alternatives to fossil fuels [1]. Improving energy efficiency and reducing consumption across all sectors is also crucial for cutting emissions [2]. Carbon pricing, through mechanisms like carbon taxes and cap-and-trade systems, effectively reduces emissions as documented by studies showing reductions between 5% and 21% [3].

Preserving natural carbon sinks such as forests, oceans, and wetlands helps sequester CO2 and contributes to emission reduction strategies [2]. On the international stage, cooperative efforts such as the Paris Agreement facilitate global collaboration on emissions targets [4]. Locally, enhancing education and promoting sustainable practices like recycling and waste reduction can foster individual and community participation. Additionally, investing in climate-smart agriculture and resilient infrastructure aids adaptation to existing climate impacts [2].

Ensuring social equity in climate strategies ensures vulnerable communities benefit from climate actions, and integrating this perspective is vital for fair and effective solutions.

Specific case studies of successful policies can offer powerful insights into effective strategies, underlining the importance of shared knowledge and collective action against climate change.
    search_queries: ['examples of successful carbon pricing policies', 'impact of renewables on national carbon footprints', 'case studies on renewable energy transitions']
    reflection: {'missing': 'Specific successful case studies of climate policies and renewable energy projects from different countries are needed to enhance credibility and provide actionable insights. Additionally, a deeper discussion on integrating social equity into climate strategies can offer a more holistic view.', 'superfluous': 'The reflection on international agreements can be reduced or specific examples of successful international collaborations can be highlighted instead, to avoid redundancy.'}
    references: ['https://hepg.hks.harvard.edu/publications/renewable-energy-sources-and-climate-change-mitigation', 'https://earthathome.org/de/climate-change-mitigation/mitigation-strategies/', 'https://www.nature.com/articles/s41467-024-48512-w', 'https://www.eea.europa.eu/en/topics/in-depth/climate-change-mitigation-reducing-emissions']

Step 6
================================= Tool Message =================================
Name: ReviseAnswer

[[{"title": "The basics of carbon pricing: a comprehensive guide - DGB Group", "url": "https://www.green.earth/carbon-pricing", "content": "International examples of successful carbon pricing implementations\nAround the world, various countries and regions have implemented successful carbon pricing initiatives. These examples serve as models of effective strategies for reducing emissions while fostering economic growth. [...] South Korea's Emissions Trading System: South Korea's ETS, launched in 2015, demonstrates the successful adoption of emissions trading as a carbon pricing mechanism in an emerging economy. [...] These international examples showcase diverse approaches to carbon pricing, illustrating how effective policies can reduce emissions while promoting economic and environmental sustainability. They provide valuable insights for governments and organisations seeking to implement carbon pricing initiatives.\nImpact on industries and businesses", "score": 0.8380581}, {"title": "Carbon Pricing for Inclusive Prosperity: The Role of Public Support", "url": "https://econfip.org/policy-briefs/carbon-pricing-for-inclusive-prosperity-the-role-of-public-support/", "content": "Three examples of successful passage and preservation of a carbon tax are Sweden, Switzerland and the Canadian Province of British Columbia.", "score": 0.83225065}, {"title": "Why is carbon pricing in some countries more successful than in ...", "url": "https://ourworldindata.org/carbon-pricing-popular", "content": "Carbon prices vary widely across existing schemes. Success stories such as that of Sweden – which currently has the highest carbon price in the world at US$139/tCO28 – demonstrate that it is indeed possible to make carbon pricing work: While the Swedish economy grew by 60% since the introduction of the Swedish carbon tax in 1991, carbon emissions decreased by 25%.9 However, the carbon price under most schemes is still lower than US$10/tCO2. [...] Political science yields two main insights regarding carbon pricing: First, ambitious carbon pricing is often correlated with high political trust and low corruption levels (see Figure 2). Cross-national studies indicate that countries with greater public distrust of politicians and perceived corruption persistently have weaker climate policies and higher greenhouse gas emissions.13,14 This is exemplified by Finland, Norway, Sweden and Switzerland, which all exhibit high levels of trust and are [...] carbon makes carbon-intensive production and consumption more expensive and creates an incentive for economic actors to turn to lower-carbon alternatives – for example, to switch from coal-based electricity generation to renewable energies. Investigations into the causal effect of carbon pricing policies, such as the European Emissions Trading System and carbon taxes in Scandinavia, show that carbon pricing is indeed a significant driver of emissions reductions.5,6,7 The mitigation effect of", "score": 0.79369855}, {"title": "These Countries Have Prices on Carbon. Are They Working?", "url": "https://www.nytimes.com/interactive/2019/04/02/climate/pricing-carbon-emissions.html", "content": "So, in 2013, Parliament enacted a carbon price floor under the system for certain sectors, including electricity, a policy that essentially functions as a carbon tax of around $25 per ton. That tax has encouraged electric utilities to rapidly switch from coal to somewhat cleaner natural gas. This is perhaps the clearest example in the world of a carbon tax leading to a significant cut in emissions.\n\nUnited States\n\n9 Northeastern states\n\nCalifornia", "score": 0.65966886}, {"title": "Carbon Pricing Examples - 7th Generation Advisors", "url": "https://7thgenerationadvisors.org/carbon-pricing-examples/", "content": "Carbon Pricing in Government. The most effective way to implement a price on carbon is through government policy and regulation because it can establish a", "score": 0.6266047}], [{"title": "The Impact of Renewable Energy on Carbon Neutrality for ... - Frontiers", "url": "https://www.frontiersin.org/journals/environmental-science/articles/10.3389/fenvs.2022.924857/full", "content": "In a similar vein, using renewable energy sources has a negative impact on the amount of carbon dioxide released into the atmosphere. This explains why a 1% significant increase in renewable energy consumption would cause a decline in emissions levels by 0.762% under one specification and 0.814% under the other specification. Renewable energy use has been shown to contribute to economic growth while reducing environmental damage in 85 developed and developing economies studied between 1991 and [...] economy that has been specified. Furthermore (Luo et al., 2019), discovered that reducing carbon dioxide emissions in the United States is only achieved in the long run when trade openness is increased. In the long run, increased trade openness has been shown to have a negative impact on carbon dioxide emissions in the United States. This was confirmed by (Gao et al., 2021). There is a correlation between increased trade openness and lower emissions, and this correlation can be explained. [...] Furthermore, (Handayani and Surachman, 2017), researched CO2 emissions have a long-term and short-term impact. According to the results of their study, renewable energy can be regarded as an important factor in reducing carbon dioxide produced during the process of power generation. This was the conclusion of their investigation into the long-term and short-term causality of the CO2 emissions. According to research carried out by Sharma et al. (2021), renewable energy can serve as a viable", "score": 0.6907474}, {"title": "Renewable energy development and carbon emissions: The role of ...", "url": "https://www.sciencedirect.com/science/article/abs/pii/S0959652624002543", "content": "This study examined the impact of renewable energy development (RED) on carbon emissions (CO2) and role of power exchange in this process.", "score": 0.6485337}, {"title": "Climate Change & the Carbon Footprint", "url": "https://www.footprintnetwork.org/our-work/climate-change/", "content": "Still, transitioning to renewable energy is one of the most powerful ways for a country to reduce its Ecological Footprint. Many countries still have a long way", "score": 0.62879556}, {"title": "The effect of renewable energy on carbon emissions through ...", "url": "https://pmc.ncbi.nlm.nih.gov/articles/PMC10907782/", "content": "The study's findings showed that while renewable energy has no appreciable impact on trade openness, it directly and negatively affects carbon emissions.", "score": 0.5999311}, {"title": "Life Cycle Assessment Harmonization | Energy Analysis - NREL", "url": "https://www.nrel.gov/analysis/life-cycle-assessment.html", "content": "The data showed that life cycle greenhouse gas (GHG) emissions from technologies powered\n                  by renewable resources are generally less than from those powered by fossil fuel-based\n                  resources. The central tendencies of all renewable technologies are between 400 and\n                  1,000 g CO2eq/kWh lower than their fossil-fueled counterparts without carbon capture and sequestration\n                  (CCS).", "score": 0.5900477}], [{"title": "Case Studies in Energy Transitions - UC Press Journals", "url": "https://online.ucpress.edu/cse/pages/cset", "content": "Two case studies are described that highlight ethical trade-offs in energy transitions. An international case study on Ethiopia and the Grand Renaissance Dam", "score": 0.8565368}, {"title": "Planning to Build Faster: A Solar Energy Case Study", "url": "https://rooseveltinstitute.org/publications/planning-to-build-faster-a-solar-energy-case-study/", "content": "Using the build-out of solar energy as a case study, this report evaluates the factors that hinder—and help—the transition to renewable energy,", "score": 0.7950386}, {"title": "IGF Case Study: Achieving a Successful Post-Mining Transition With ...", "url": "https://www.iisd.org/publications/igf-case-study-post-mining-transition-renewable-energy", "content": "This report examines how two former mine sites were converted into spaces generating renewable energy in British Columbia, Canada and Ruhr, Germany.", "score": 0.756376}, {"title": "Case Studies of Renewable Thermal Energy", "url": "https://www.c2es.org/document/case-studies-of-renewable-thermal-energy/", "content": "This series of case studies showcases successful outcomes from the use of renewable thermal technologies at several different large companies and in a major", "score": 0.70483154}, {"title": "Case studies - Sustainable Energy for All | SEforALL", "url": "https://www.seforall.org/case-studies", "content": "The Malawi Integrated Energy Planning Tool offers in-depth data and insights to inform the country’s efforts to advance the sustainable development of clean energy solutions to close the energy access gap.\n\nThe Nigeria Energy Transition Plan has been adopted as national policy and is the framework for the country to achieve universal energy access and net-zero emissions. [...] The 24/7 Carbon-Free Energy Compact has solicited commitments from over 116 organizations and governments to decarbonize their electricity consumption.\n\nA new government-backed Energy Transition and Investment Plan will lay out strategies and policies needed to reach Barbados’ climate and economic objectives in concert. [...] The Sierra Leone Healthcare Electrification Project has already deployed solar PV and battery storage systems to improve the reliability of electricity supplies at six hospitals, with more health facilities to come.\n\nFooter menu", "score": 0.6899958}]]


Step 7
================================== Ai Message ==================================
Tool Calls:
  ReviseAnswer (call_QEIINip9joGSjeyzB3ioangm)
 Call ID: call_QEIINip9joGSjeyzB3ioangm
  Args:
    answer: Addressing the climate crisis involves multi-level strategies. Transitioning to renewable energy like solar, wind, and hydropower is essential, as these sources significantly reduce greenhouse gas emissions and provide sustainable alternatives to fossil fuels [1]. Improving energy efficiency and reducing consumption across all sectors is also crucial for cutting emissions [2]. Carbon pricing, through mechanisms like carbon taxes and cap-and-trade systems, effectively reduces emissions as documented by studies showing reductions between 5% and 21% [3].

Preserving natural carbon sinks such as forests, oceans, and wetlands helps sequester CO2 and contributes to emission reduction strategies [2]. On the international stage, cooperative efforts such as the Paris Agreement facilitate global collaboration on emissions targets [4]. Locally, enhancing education and promoting sustainable practices like recycling and waste reduction can foster individual and community participation. Additionally, investing in climate-smart agriculture and resilient infrastructure aids adaptation to existing climate impacts [2].

Ensuring social equity in climate strategies ensures vulnerable communities benefit from climate actions, and integrating this perspective is vital for fair and effective solutions.

Specific case studies of successful policies can offer powerful insights into effective strategies, underlining the importance of shared knowledge and collective action against climate change.
    search_queries: ['examples of successful carbon pricing policies', 'impact of renewables on national carbon footprints', 'case studies on renewable energy transitions']
    reflection: {'missing': 'Specific successful case studies of climate policies and renewable energy projects from different countries are needed to enhance credibility and provide actionable insights. Additionally, a deeper discussion on integrating social equity into climate strategies can offer a more holistic view.', 'superfluous': 'The reflection on international agreements can be reduced or specific examples of successful international collaborations can be highlighted instead, to avoid redundancy.'}
    references: ['https://hepg.hks.harvard.edu/publications/renewable-energy-sources-and-climate-change-mitigation', 'https://earthathome.org/de/climate-change-mitigation/mitigation-strategies/', 'https://www.nature.com/articles/s41467-024-48512-w', 'https://www.eea.europa.eu/en/topics/in-depth/climate-change-mitigation-reducing-emissions']
"""

In [7]:
pip install schema

In [12]:
import datetime
import json
import os
from typing import List, Annotated, Literal
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

# 최신 패키지 임포트
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.messages import HumanMessage, ToolMessage, BaseMessage
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import StructuredTool
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# --- [중요] API 키 설정 ---
# os.environ["TAVILY_API_KEY"] = "your_tavily_api_key"
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_hf_token"

# --- 1. Schema 정의 ---
class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous.")

class AnswerQuestion(BaseModel):
    """Answer the question."""
    answer: str = Field(description="~250 word detailed answer.")
    reflection: Reflection = Field(description="Your reflection on the answer.")
    search_queries: List[str] = Field(description="1-3 search queries for research.")

class ReviseAnswer(AnswerQuestion):
    """Revise the answer based on critique."""
    references: List[str] = Field(description="Numerical citations used in the answer.")

# --- 2. Retry 로직 정의 ---
class ResponderWithRetries:
    def __init__(self, runnable, validator):
        self.runnable = runnable
        self.validator = validator

    def respond(self, state: TypedDict):
        response = self.runnable.invoke(state)
        return {"messages": [response]}

# --- 3. Actor 및 모델 설정 ---
actor_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are expert researcher. Current time: {time}
    1. {first_instruction}
    2. Reflect and critique your answer. Be severe to maximize improvement.
    3. Recommend search queries to research information."""),
    MessagesPlaceholder(variable_name="messages"),
    ("user", "\n\n<system>Respond using the {function_name} function.</system>"),
]).partial(time=lambda: datetime.datetime.now().isoformat())

# Hugging Face 모델 설정
llm_ep = HuggingFaceEndpoint(repo_id="Qwen/Qwen2.5-7B-Instruct", task="text-generation")
llm = ChatHuggingFace(llm=llm_ep)

# --- 4. Chain 및 도구 설정 (수정된 부분) ---
validator = PydanticToolsParser(tools=[AnswerQuestion])
initial_answer_chain = actor_prompt_template.partial(
    first_instruction="Provide a detailed ~250 word answer.",
    function_name=AnswerQuestion.__name__,
) | llm.bind_tools(tools=[AnswerQuestion])

first_responder = ResponderWithRetries(runnable=initial_answer_chain, validator=validator)

revise_instructions = """Revise your previous answer. Include numerical citations and a References section."""
revision_chain = actor_prompt_template.partial(
    first_instruction=revise_instructions,
    function_name=ReviseAnswer.__name__,
) | llm.bind_tools(tools=[ReviseAnswer])
revisor = ResponderWithRetries(runnable=revision_chain, validator=PydanticToolsParser(tools=[ReviseAnswer]))

# Tavily 도구 초기화
tavily_tool = TavilySearchResults(max_results=5)

# [해결] Docstring이 추가된 실행 함수
def run_queries(search_queries: list[str], **kwargs):
    """
    Run the generated search queries using Tavily to gather real-time information.
    This function takes a list of query strings and returns gathered search results.
    """
    return tavily_tool.batch([{"query": query} for query in search_queries])

# ToolNode 생성
tool_node = ToolNode([
    StructuredTool.from_function(run_queries, name=AnswerQuestion.__name__),
    StructuredTool.from_function(run_queries, name=ReviseAnswer.__name__),
])

# --- 5. Graph 구축 ---
class State(TypedDict):
    messages: Annotated[list, add_messages]

def _get_num_iterations(messages):
    # 도구 호출 횟수를 계산하여 무한 루프 방지
    return sum(1 for m in messages if hasattr(m, 'tool_calls') and m.tool_calls)

def event_loop(state: State):
    num_iterations = _get_num_iterations(state["messages"])
    if num_iterations > 3:
        return END
    return "execute_tools"

builder = StateGraph(State)
builder.add_node("draft", first_responder.respond)
builder.add_node("execute_tools", tool_node)
builder.add_node("revise", revisor.respond)

builder.add_edge(START, "draft")
builder.add_edge("draft", "execute_tools")
builder.add_edge("execute_tools", "revise")
builder.add_conditional_edges("revise", event_loop, ["execute_tools", END])

graph = builder.compile()

# --- 6. 실행 ---


print("--- 에이전트 실행 시작 ---")
events = graph.stream(
    {"messages": [("user", "How should we handle the climate crisis?")]},
    stream_mode="values",
)
for i, step in enumerate(events):
    print(f"\n[Step {i}]")
    step["messages"][-1].pretty_print()

--- 에이전트 실행 시작 ---

[Step 0]
================================ Human Message =================================

How should we handle the climate crisis?

[Step 1]
================================== Ai Message ==================================
Tool Calls:
  AnswerQuestion (call_8c37otib9b1u4brrbwr3e1cx)
 Call ID: call_8c37otib9b1u4brrbwr3e1cx
  Args:
    answer: Addressing the climate crisis requires a multifaceted approach involving global cooperation, policy changes, technological advancements, and individual actions. Nationally, governments must implement stringent policies to reduce greenhouse gas emissions, such as carbon pricing, renewable energy mandates, and energy efficiency standards. Internationally, countries need to adhere to and strengthen commitments under the Paris Agreement. Technological innovations, including renewable energy sources like solar and wind, and carbon capture technologies, are crucial. Additionally, reforestation and sustainable land management can help 

# Language Agent Tree Search

In [13]:
pip install langchain-anthropic==0.3.10 langchain-community==0.3.21 langchain-openai==0.3.12 langgraph==0.3.25 langsmith==0.3.24 tavily-python==0.5.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 4.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 5.6 MB/

In [3]:
import math
from collections import deque
from typing import Optional

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage

from pydantic import BaseModel, Field

In [4]:
class Reflection(BaseModel):
    reflections: str = Field(
        description="The critique and reflections on the sufficiency, superfluency,"
        " and general quality of the response"
    )
    score: int = Field(
        description="Score from 0-10 on the quality of the candidate response.",
        gte=0,
        lte=10,
    ) # type: ignore
    found_solution: bool = Field(
        description="Whether the response has fully solved the question or task."
    )

    def as_message(self):
        return HumanMessage(
            content=f"Reasoning: {self.reflections}\nScore: {self.score}"
        )

    @property
    def normalized_score(self) -> float:
        return self.score / 10.0

class Node:
    def __init__(
        self,
        messages: list[BaseMessage],
        reflection: Reflection,
        parent: Optional["Node"] = None,
    ):
        self.messages = messages
        self.parent = parent
        self.children = []
        self.value = 0
        self.visits = 0
        self.reflection = reflection
        self.depth = parent.depth + 1 if parent is not None else 1
        self._is_solved = reflection.found_solution if reflection else False
        if self._is_solved:
            self._mark_tree_as_solved()
        self.backpropagate(reflection.normalized_score)

    def __repr__(self) -> str:
        return (
            f"<Node value={self.value}, visits={self.visits},"
            f" solution={self.messages} reflection={self.reflection}/>"
        )

    @property
    def is_solved(self):
        """If any solutions exist, we can end the search."""
        return self._is_solved

    @property
    def is_terminal(self):
        return not self.children

    @property
    def best_child_score(self):
        """Return the child with the highest value."""
        if not self.children:
            return None
        return max(self.children, key=lambda child: int(child.is_solved) * child.value)

    @property
    def height(self) -> int:
        """Check for how far we've rolled out the tree."""
        if self.children:
            return 1 + max([child.height for child in self.children])
        return 1

    def upper_confidence_bound(self, exploration_weight=1.0):
        """Return the UCT score. This helps balance exploration vs. exploitation of a branch."""
        if self.parent is None:
            raise ValueError("Cannot obtain UCT from root node")
        if self.visits == 0:
            return self.value
        # Encourages exploitation of high-value trajectories
        average_reward = self.value / self.visits
        # Encourages exploration of less-visited trajectories
        exploration_term = math.sqrt(math.log(self.parent.visits) / self.visits)
        return average_reward + exploration_weight * exploration_term

    def backpropagate(self, reward: float):
        """Update the score of this node and its parents."""
        node = self
        while node:
            node.visits += 1
            node.value = (node.value * (node.visits - 1) + reward) / node.visits
            node = node.parent

    def get_messages(self, include_reflections: bool = True):
        if include_reflections:
            return self.messages + [self.reflection.as_message()]
        return self.messages

    def get_trajectory(self, include_reflections: bool = True) -> list[BaseMessage]:
        """Get messages representing this search branch."""
        messages = []
        node = self
        while node:
            messages.extend(
                node.get_messages(include_reflections=include_reflections)[::-1]
            )
            node = node.parent
        # Reverse the final back-tracked trajectory to return in the correct order
        return messages[::-1]  # root solution, reflection, child 1, ...

    def _get_all_children(self):
        all_nodes = []
        nodes = deque()
        nodes.append(self)
        while nodes:
            node = nodes.popleft()
            all_nodes.extend(node.children)
            for n in node.children:
                nodes.append(n)
        return all_nodes

    def get_best_solution(self):
        """Return the best solution from within the current sub-tree."""
        all_nodes = [self] + self._get_all_children()
        best_node = max(
            all_nodes,
            # We filter out all non-terminal, non-solution trajectories
            key=lambda node: int(node.is_terminal and node.is_solved) * node.value,
        )
        return best_node

    def _mark_tree_as_solved(self):
        parent = self.parent
        while parent:
            parent._is_solved = True
            parent = parent.parent

/tmp/ipython-input-3909326980.py:6: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'gte', 'lte'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  score: int = Field(


In [5]:
from typing_extensions import TypedDict

class TreeState(TypedDict):
    # The full tree
    root: Node
    # The original input
    input: str

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode

tavily_tool = TavilySearchResults(max_results=5)
tools = [tavily_tool]
tool_node = ToolNode(tools=tools)


from langchain_core.output_parsers.openai_tools import (
    JsonOutputToolsParser,
    PydanticToolsParser,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import chain as as_runnable

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Reflect and grade the assistant response to the user question below.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="candidate"),
    ]
)

reflection_llm_chain = (
    prompt
    | llm.bind_tools(tools=[Reflection], tool_choice="Reflection").with_config(
        run_name="Reflection"
    )
    | PydanticToolsParser(tools=[Reflection])
)


@as_runnable
def reflection_chain(inputs) -> Reflection:
    tool_choices = reflection_llm_chain.invoke(inputs)
    reflection = tool_choices[0]
    if not isinstance(inputs["candidate"][-1], AIMessage):
        reflection.found_solution = False
    return reflection

In [14]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm_ep = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    max_new_tokens=1024,
    temperature=0.7,
    # n=1 속성이 있다면 명시적으로 추가 (서버마다 다를 수 있음)
)
llm = ChatHuggingFace(llm=hf_ep)

from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode

tavily_tool = TavilySearchResults(max_results=5)
tools = [tavily_tool]
tool_node = ToolNode(tools=tools)


from langchain_core.output_parsers.openai_tools import (
    JsonOutputToolsParser,
    PydanticToolsParser,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import chain as as_runnable

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Reflect and grade the assistant response to the user question below.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="candidate"),
    ]
)

reflection_llm_chain = (
    prompt
    | llm.bind_tools(tools=[Reflection], tool_choice="Reflection").with_config(
        run_name="Reflection"
    )
    | PydanticToolsParser(tools=[Reflection])
)


@as_runnable
def reflection_chain(inputs) -> Reflection:
    tool_choices = reflection_llm_chain.invoke(inputs)
    reflection = tool_choices[0]
    if not isinstance(inputs["candidate"][-1], AIMessage):
        reflection.found_solution = False
    return reflection

In [15]:
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnableConfig

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI assistant.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)


initial_answer_chain = prompt_template | llm.bind_tools(tools=tools).with_config(
    run_name="GenerateInitialCandidate"
)

parser = JsonOutputToolsParser(return_id=True)
initial_response = initial_answer_chain.invoke(
    {"input": "Write a research report on lithium pollution."}
)
initial_response

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"lithium pollution environmental impact lithium mining contamination water"}', 'name': 'tavily_search_results_json', 'description': None}, 'id': 'fc_1b52a364-512b-4540-b75d-4c8e2e29cad7', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 194, 'prompt_tokens': 171, 'total_tokens': 365}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_92d51d08e5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--df19d701-971b-41ae-b160-f84fd194d741-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'lithium pollution environmental impact lithium mining contamination water'}, 'id': 'fc_1b52a364-512b-4540-b75d-4c8e2e29cad7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 171, 'output_tokens': 194, 'total_tokens': 365})

In [16]:
# Define the node we will add to the graph
def generate_initial_response(state: TreeState) -> dict:
    """Generate the initial candidate response."""
    res = initial_answer_chain.invoke({"input": state["input"]})
    parsed = parser.invoke(res)
    tool_responses = [
        tool_node.invoke(
            {
                "messages": [
                    AIMessage(
                        content="",
                        tool_calls=[
                            {"name": r["type"], "args": r["args"], "id": r["id"]}
                        ],
                    )
                ]
            }
        )
        for r in parsed
    ]
    output_messages = [res] + [tr["messages"][0] for tr in tool_responses]
    reflection = reflection_chain.invoke(
        {"input": state["input"], "candidate": output_messages}
    )
    root = Node(output_messages, reflection=reflection)
    return {
        **state,
        "root": root,
    }

In [17]:
# This generates N candidate values
# for a single input to sample actions from the environment
def generate_candidates(messages: ChatPromptValue, config: RunnableConfig):
    n = config["configurable"].get("N", 5)
    bound_kwargs = llm.bind_tools(tools=tools).kwargs
    chat_result = llm.generate(
        [messages.to_messages()],
        n=n,
        callbacks=config["callbacks"],
        run_name="GenerateCandidates",
        **bound_kwargs,
    )
    return [gen.message for gen in chat_result.generations[0]]


expansion_chain = prompt_template | generate_candidates

In [ ]:
res = expansion_chain.invoke({"input": "Write a research report on lithium pollution."})
res

In [ ]:
from collections import defaultdict


def select(root: Node) -> dict:
    """Starting from the root node a child node is selected at each tree level until a leaf node is reached."""

    if not root.children:
        return root

    node = root
    while node.children:
        max_child = max(node.children, key=lambda child: child.upper_confidence_bound())
        node = max_child

    return node


def expand(state: TreeState, config: RunnableConfig) -> dict:
    """Starting from the "best" node in the tree, generate N candidates for the next step."""
    root = state["root"]
    best_candidate: Node = select(root)
    messages = best_candidate.get_trajectory()
    # Generate N candidates from the single child candidate
    new_candidates = expansion_chain.invoke(
        {"input": state["input"], "messages": messages}, config
    )
    parsed = parser.batch(new_candidates)
    flattened = [
        (i, tool_call)
        for i, tool_calls in enumerate(parsed)
        for tool_call in tool_calls
    ]
    tool_responses = [
        (
            i,
            tool_node.invoke(
                {
                    "messages": [
                        AIMessage(
                            content="",
                            tool_calls=[
                                {
                                    "name": tool_call["type"],
                                    "args": tool_call["args"],
                                    "id": tool_call["id"],
                                }
                            ],
                        )
                    ]
                }
            ),
        )
        for i, tool_call in flattened
    ]
    collected_responses = defaultdict(list)
    for i, resp in tool_responses:
        collected_responses[i].append(resp["messages"][0])
    output_messages = []
    for i, candidate in enumerate(new_candidates):
        output_messages.append([candidate] + collected_responses[i])

    # Reflect on each candidate
    # For tasks with external validation, you'd add that here.
    reflections = reflection_chain.batch(
        [{"input": state["input"], "candidate": msges} for msges in output_messages],
        config,
    )
    # Grow tree
    child_nodes = [
        Node(cand, parent=best_candidate, reflection=reflection)
        for cand, reflection in zip(output_messages, reflections)
    ]
    best_candidate.children.extend(child_nodes)
    # We have already extended the tree directly, so we just return the state
    return state

In [ ]:
from typing import Literal

from langgraph.graph import END, StateGraph, START


def should_loop(state: TreeState):
    """Determine whether to continue the tree search."""
    root = state["root"]
    if root.is_solved:
        return END
    if root.height > 5:
        return END
    return "expand"


builder = StateGraph(TreeState)
builder.add_node("start", generate_initial_response)
builder.add_node("expand", expand)
builder.add_edge(START, "start")


builder.add_conditional_edges(
    "start",
    # Either expand/rollout or finish
    should_loop,
    ["expand", END],
)
builder.add_conditional_edges(
    "expand",
    # Either continue to rollout or finish
    should_loop,
    ["expand", END],
)

graph = builder.compile()

In [ ]:
question = "Generate a table with the average size and weight, as well as the oldest recorded instance for each of the top 5 most common birds."
last_step = None
for step in graph.stream({"input": question}):
    last_step = step
    step_name, step_state = next(iter(step.items()))
    print(step_name)
    print("rolled out: ", step_state["root"].height)
    print("---")